In [1]:
# 모듈 로딩
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np
import matplotlib as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [2]:
# 데이터 로딩
DATA_PATH = '../Dataset/팀원 데이터/rpm170_KNN_n7_29_31.csv'
dataDF = pd.read_csv(DATA_PATH)

### train_test 나누기

In [3]:
dataDF

,c_temp_pv,k_rpm_pv,n_temp_pv,scale_pv,s_temp_pv
0,69.6,189.0,67.2,3.010000,67.1
1,69.8,189.0,67.2,3.010000,67.0
2,69.7,189.0,67.9,3.080000,65.9
3,69.7,189.0,67.8,3.080000,65.9
4,69.7,189.0,67.8,3.080000,65.9
...,...,...,...,...,...
139778,70.0,169.0,69.8,3.065714,70.2
139779,70.1,169.0,69.7,3.080000,70.3
139780,70.1,169.0,69.7,3.072857,70.4
139781,70.1,169.0,69.6,3.072857,70.5


In [4]:
target = dataDF['scale_pv']
feature = dataDF.drop(columns='scale_pv')

In [5]:
target

0         3.010000
1         3.010000
2         3.080000
3         3.080000
4         3.080000
            ...   
139778    3.065714
139779    3.080000
139780    3.072857
139781    3.072857
139782    3.071429
Name: scale_pv, Length: 139783, dtype: float64

In [6]:
feature

,c_temp_pv,k_rpm_pv,n_temp_pv,s_temp_pv
0,69.6,189.0,67.2,67.1
1,69.8,189.0,67.2,67.0
2,69.7,189.0,67.9,65.9
3,69.7,189.0,67.8,65.9
4,69.7,189.0,67.8,65.9
...,...,...,...,...
139778,70.0,169.0,69.8,70.2
139779,70.1,169.0,69.7,70.3
139780,70.1,169.0,69.7,70.4
139781,70.1,169.0,69.6,70.5


In [7]:
X_train, X_test, y_train, y_test = train_test_split(feature,target, test_size= 0.2, random_state=77)

In [8]:
# Bayesian Optimization을 위한 목적 함수 정의
def xgb_evaluate(max_depth, n_estimators, learning_rate, colsample_bytree, subsample, min_child_weight):
    params = {
        'max_depth': int(max_depth),  # 정수형으로 변환
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),  # 정수형으로 변환
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'min_child_weight': int(min_child_weight),  # 정수형으로 변환
        'objective': 'reg:squarederror',
        'random_state': 2,
        'n_jobs': 4
    }
    model = XGBRegressor(**params)
    cv_scores = cross_val_score(model, X_train.values, y_train, cv=3, scoring='neg_mean_squared_error')
    return -cv_scores.mean()  # MSE를 최소화해야 하므로 부호 반전

## MSE는 낮을수록 좋은 지표이므로, 부호를 반전하여 반환
## Bayesian Optimization은 목적 함수가 최대화하는 방향으로 작동하기 때문에, MSE의 부호를 반전하여 "최적화 대상 점수"로 사용


In [9]:
# Bayesian Optimization 설정
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds={
        'max_depth': (3, 7),
        'n_estimators': (500, 1000),
        'learning_rate': (0.03, 0.1),
        'colsample_bytree': (0.5, 0.7),
        'subsample': (0.5, 0.8),
        'min_child_weight': (1, 5),
    },
    random_state=2,
)

In [10]:
# 최적화 실행
# init_points는 초기 random으로 몇개의 값을 찾을것인지(베이지안 최적화는 기존 데이터를 기반으로 가우시안 프로세스 모델)
# n_iter는 추가로 조사할 값들을 몇개 찾을지 설정(어디를 샘플링하면 최대값을 잘 찾을 수 있을지를 판단)
optimizer.maximize(init_points=5, n_iter=25)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | 0.0003078 | 0.5872    | 0.03181   | 5.199     | 2.741     | 710.2     | 0.5991    |
| 2         | 0.000298  | 0.5409    | 0.07335   | 4.199     | 2.067     | 810.6     | 0.6587    |
| 3         | 0.0003236 | 0.5269    | 0.06595   | 3.738     | 4.141     | 927.0     | 0.6483    |
| 4         | 0.0003036 | 0.6693    | 0.03558   | 5.021     | 1.261     | 714.1     | 0.529     |
| 5         | 0.0003322 | 0.5254    | 0.07177   | 3.904     | 1.428     | 610.2     | 0.6049    |
| 6         | 0.0003462 | 0.5958    | 0.04168   | 3.183     | 1.021     | 626.4     | 0.7675    |
| 7         | 0.0002902 | 0.529     | 0.0623    | 5.704     | 4.542     | 649.7     | 0.7858    |
| 8         | 0.0002696 | 0.5452    | 0.09424   | 6.637     | 4.749     | 620.9     | 0.5344    |
| 9         | 0.0003

In [11]:
# 최적의 하이퍼파라미터 추출
best_params = optimizer.max['params']

# 하이퍼파라미터 정수형 변환 (필요 시)
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['min_child_weight'] = int(best_params['min_child_weight'])

In [12]:
# 최종 모델 생성 및 학습
final_model = XGBRegressor(
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    n_estimators=best_params['n_estimators'],
    colsample_bytree=best_params['colsample_bytree'],
    subsample=best_params['subsample'],
    min_child_weight=best_params['min_child_weight'],
    objective='reg:squarederror',
    random_state=2,
    n_jobs=4
)

final_model.fit(X_train.values, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5795117600809843,
             enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.032543501006192146,
             max_delta_step=0, max_depth=3, min_child_weight=4, missing=nan,
             monotone_constraints='()', n_estimators=614, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=2, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.5752190694540291,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
# 모델 저장
final_model.save_model('xgb_rpm170_2.9 ~ 3.1.json')

In [14]:
loaded_model = XGBRegressor()
loaded_model.load_model('xgb_rpm170_2.9 ~ 3.1.json')

# 로드한 모델로 예측
y_pred = loaded_model.predict(X_test)

y_pred = loaded_model.predict(X_test)
MAE = mean_absolute_error(y_test,y_pred)
MAPE = mean_absolute_percentage_error(y_test,y_pred)
Score = loaded_model.score(X_test,y_test)
print(f'MAE => {MAE}     MAPE => {MAPE*100}    R2 => {Score}')

MAE => 0.013937999829832294     MAPE => 0.4589607751950806    R2 => 0.19971533203104463
